## Using APIs in Projects


When getting data from APIs, I strongly suggest following a three-step workflow:

1. Write some code that gets data from an API and saves all of the data (if possible) to a file
2. Write a second program (usually a second file) that loads the data from the API, extracts the data that will be useful for analysis, and saves it in a flat file (typically a CSV).
3. Program number 3 loads the CSV file and does the analysis

This approach has a few important benefits.

The first and most important is that often it is difficult to get the same raw data again. If you are using Twitter, then the Search API only lets you get the last week. If you are doing analysis a month down the road and decide that you really wish you had saved metadata about the number of retweets, it is too late. By saving the raw data you can change your measures or analysis strategy and still have access to the data.

The second is that this gives you a nice pipeline, with intermediate files. Instead of including the entire raw data file in the code that does analysis, you only have to load the CSV, which is often much smaller and easier to work with.

This brief lesson will show an example of this workflow, using `tweepy`.

Note that I'm going to put everything in one file for convenience, but my typical workflow is to put these in separate files and then run each file separately.

## Program 1 - Data Retrieval

The goal of our project is to produce a visualization of the histogram of the number of retweets for recent tweets about President Trump. The first program gets tweets about President Trump.

Older versions of tweepy made it easier to get the JSON returned by 

In [ ]:
import tweepy
import json
import requests
from twitter_authentication import bearer_token
import pandas as pd
import time

client = tweepy.Client(bearer_token, 
                       return_type=dict, # Without this, tweepy returns an object-based response
                       wait_on_rate_limit=True
                      )

ModuleNotFoundError: ignored

In [ ]:
with open('raw_trump_tweets.txt', 'w') as f: # By opening with 'a' (for append) we can write to this file again
                                            # e.g., if we need to restart data collection.
    num_tweets = 4000
    tweet_count = 0
    next_token = None
    
    while tweet_count < num_tweets:
        response = client.search_recent_tweets(
            query='Trump -is:retweet', 
            user_fields = ['username', 'public_metrics', 'description', 'location'],
            tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
            expansions = 'author_id',
            next_token = next_token,
            max_results=100)
        
        next_token = response['meta']['next_token'] # get the "next token", which gives us the next page
        tweets = response['data']
        tweet_count += len(tweets)
        
        f.write(json.dumps(response)+ '\n') # Write each response's JSON as a new line
        time.sleep(.5)

## Program 2 - Data Cleaning

This program loads the saved raw data, grabs what we want, and converts it into a pandas dataframe and a CSV.

I decided to save the timestamp, text, and retweet and favorite counts, together with the public metrics about the author. Tweepy puts author info in an "includes" dictionary, so we have to extract it from there.

This is also where you typically would do more complicated measure creation. Here I show how to create a measure of tweet_length.

In [ ]:
with open('raw_trump_tweets.txt', 'r') as f:
    responses = []
    for line in f.readlines():
        responses.append(json.loads(line))

In [ ]:
result = []
user_dict = {}
# Loop through each response object
for response in responses:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response['includes']['users']:
        user_dict[user['id']] = {'username': user['username'], 
                              'followers': user['public_metrics']['followers_count'],
                              'tweets': user['public_metrics']['tweet_count']
                             }
    for tweet in response['data']:
        # For each tweet, find the author's information
        author_info = user_dict[tweet['author_id']]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet['author_id'], 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'text': tweet['text'],
                       'created_at': tweet['created_at'],
                       'retweets': tweet['public_metrics']['retweet_count'],
                       'likes': tweet['public_metrics']['like_count'],
                       'tweet_length': len(tweet['text'])
                      })
df = pd.DataFrame(result)
df.to_csv('cleaned_data.csv', index=False)

## Program 3 - Data Analysis

Here we use pandas to load the data and analyze it. This could include statistical tests. Here, I'm just visualizing the distribution of retweets and the relationship between retweets and length.

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('./cleaned_data.csv')

In [ ]:
# Just make sure it looks OK.
df.sort_values('retweets')

In [ ]:
sns.displot(df.retweets)

As expected, it's super skewed, with most tweets never getting retweeted while a few get tons of retweets.

Let's see if it changes if we get rid of the tweets that never got retweeted (like, maybe we have a principled reason to believe they are different than other tweets).

In [ ]:
sns.displot(df.loc[df.retweets > 1, 'retweets']);

As I thought, this is a somewhat "scale-free" distribution, meaning wherever you zoom in, you see the same pattern. Try changing the `0` up above to any (small) number.

For fun, let's also look at the relationship between retweets and tweet length.

In [ ]:
import numpy as np

In [ ]:
sns.jointplot(y='retweets', x='tweet_length', data = df);

In [ ]:
# Because retweets are so skewed, let's log them
p = sns.jointplot(y=np.log(df.retweets + 1), x='tweet_length', data = df)
p.set_axis_labels('Tweet Length','Retweets (log)');